In [1]:
using LinearAlgebra, Symbolics

In [2]:
@variables r s;

In [32]:
#=
using DelimitedFiles
datos = 10*ones(100,5) + 5*randn(100,5);
writedlm("miarchivo.txt", datos)
=#

In [3]:
ν = 0.3 # Coeficiente de Poisson
EM = 2.1E06 # Módulo de Young - kgf/cm^2
C = (EM/(1-ν^2)) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2]; # Matriz de constantes elásticas (Tensión plana)

In [4]:
NodalCoords = [0 0; 3 0; 1.5 3; 1.5 0; 2.25 1.5; 0.75 1.5] # Coordenadas Globales para elemento de 6 nodos

6×2 Matrix{Float64}:
 0.0   0.0
 3.0   0.0
 1.5   3.0
 1.5   0.0
 2.25  1.5
 0.75  1.5

In [5]:
# Funciones de interpolación para un elemento isoparamétrico triangular de 6 nodos
h6 = 4*s*(1-r-s)
h5 = 4*r*s
h4 = 4*r*(1-r-s)
h1 = 1-r-s-1/2*h4-1/2*h6
h2 = r-1/2*h4-1/2*h5
h3 = s-1/2*h5-1/2*h6;

In [6]:
Dr = Differential(r)
Ds = Differential(s)

(::Differential) (generic function with 2 methods)

In [7]:
H = [h1 h2 h3 h4 h5 h6]
H1 = [h1 h2 h3 h4 h5 h6; h1 h2 h3 h4 h5 h6]
HM = [h1 0 h2 0 h3 0 h4 0 h5 0 h6 0; 0 h1 0 h2 0 h3 0 h4 0 h5 0 h6]
display(H)

1×6 Matrix{Num}:
 1 - r - s - 2.0r*(1 - r - s) - 2.0s*(1 - r - s)  …  4r*s  4s*(1 - r - s)

In [8]:
∇H = [transpose([expand_derivatives.(Dr(H1[1,i])) for i in 1:6]); transpose([expand_derivatives.(Ds(H1[2,i])) for i in 1:6])]

2×6 Matrix{Num}:
 2.0r + 2.0s - 1 - 2.0(1 - r - s)  …  4s                -4s
 2.0r + 2.0s - 1 - 2.0(1 - r - s)     4r  4(1 - r - s) - 4s

In [9]:
x = expand((H * NodalCoords[:,1])[1])
y = expand((H * NodalCoords[:,2])[1])
display(x)
display(y)

1.5s + 3.0r

3.0s

In [10]:
dxdr = expand_derivatives(Dr(x))
dxds = expand_derivatives(Ds(x))        
dydr = expand_derivatives(Dr(y))
dyds = expand_derivatives(Ds(y));

In [11]:
J = [dxdr dydr; dxds dyds] # Jacobiano
J_inv = inv(J) # Inversa del Jacobiano
J_det = det(J) # Determinante del Jacobiano
display(J)
display(J_inv)
display(J_det)

2×2 Matrix{Num}:
 3.0  0
 1.5  3.0

2×2 Matrix{Num}:
  0.333333  0.0
 -0.166667  0.333333

9.0

In [12]:
B1 = transpose(∇H) * J_inv

6×2 Matrix{Num}:
  0.166667(2.0r + 2.0s - 1 - 2.0(1 - r - s))  …   0.333333(2.0r + 2.0s - 1 - 2.0(1 - r - s))
  0.333333(1 + 2.0r - 2.0s - 2.0(1 - r - s))      0.0
 -0.166667(1 + 2.0s - 2.0r - 2.0(1 - r - s))      0.333333(1 + 2.0s - 2.0r - 2.0(1 - r - s))
  0.333333(4(1 - r - s) - 4r) + 0.666667r        -1.33333r
  1.33333s - 0.666667r                            1.33333r
 -1.33333s - 0.166667(4(1 - r - s) - 4s)      …   0.333333(4(1 - r - s) - 4s)

In [13]:
B = [B1[1,1] 0 B1[2,1] 0 B1[3,1] 0 B1[4,1] 0 B1[5,1] 0 B1[6,1] 0
    0 B1[1,2] 0 B1[2,2] 0 B1[3,2] 0 B1[4,2] 0 B1[5,2] 0 B1[6,2]
    B1[1,2] B1[1,1] B1[2,2] B1[2,1] B1[3,2] B1[3,1] B1[4,2] B1[4,1] B1[5,2] B1[5,1] B1[6,2] B1[6,1]]

3×12 Matrix{Num}:
 0.166667(2.0r + 2.0s - 1 - 2.0(1 - r - s))  …   0
 0                                               0.333333(4(1 - r - s) - 4s)
 0.333333(2.0r + 2.0s - 1 - 2.0(1 - r - s))     -1.33333s - 0.166667(4(1 - r - s) - 4s)

In [29]:
K = zeros(12,12) 

#Integramos por medio de cuadratura de Gauss para siete puntos de integración sobre dominios triangulares
r1 = 0.1012865073235
r2 = 0.7974269853531
r3 = r1
r4 = 0.4701420641051
r5 = r4
r6 = 0.0597158717898
r7 =  1/3

s1 = r1
s2 = r1
s3 = r2
s4 = r6
s5 = r4
s6 = r4
s7 = r7

w1 = 0.1259391805448
w2 = w1
w3 = w1
w4 = 0.1323941527885
w5 = w4
w6 = w4
w7 = 0.225

GL7T = [r1 s1; r2 s2; r3 s3; r4 s4; r5 s5; r6 s6; r7 s7] # Puntos de Gauss para integración
#display(GL7T)
WGL7T = [w1 w2 w3 w4 w5 w6 w7]

for i in 1:7
    Jg = substitute(J, Dict([r => GL7T[i,1], s => GL7T[i,2]]))        
    Jg_inv = substitute(J_inv, Dict([r => GL7T[i,1], s => GL7T[i,2]]))
    Jg_det = substitute(J_det, Dict([r => GL7T[i,1], s => GL7T[i,2]]))
                
    global Bg = substitute(B, Dict([r => GL7T[i,1], s => GL7T[i,2]]))
    
    K += 1/2*(transpose(Bg) * C * Bg * Jg_det * WGL7T[i])
        
end
display(B)
display(Bg)
display(K)

3×12 Matrix{Num}:
 0.166667(2.0r + 2.0s - 1 - 2.0(1 - r - s))  …   0
 0                                               0.333333(4(1 - r - s) - 4s)
 0.333333(2.0r + 2.0s - 1 - 2.0(1 - r - s))     -1.33333s - 0.166667(4(1 - r - s) - 4s)

3×12 Matrix{Num}:
 -0.0555556   0          0.111111  …  0         -0.444444     0
  0          -0.111111   0            0.444444   0            1.4803e-16
 -0.111111   -0.0555556  0.0          0.222222   1.4803e-16  -0.444444

12×12 Matrix{Num}:
      6.92308e5   375000.0         …      -1.53846e5    38461.5
 375000.0              1.25481e6      -38461.5             -1.40385e6
      1.92308e5        1.15385e5          -1.67449e-7      -1.74217e-8
      1.34615e5    67307.7                -2.03301e-8      -5.86151e-8
  38461.5           9615.38               -1.53846e5   -38461.5
  -9615.38             3.50962e5   …   38461.5             -1.40385e6
     -7.69231e5       -4.61538e5          -1.53846e6        1.0e6
     -5.38462e5       -2.69231e5           1.0e6           -5.38462e5
      2.13186e-8       1.15469e-8         -1.53846e6       -1.0e6
      1.15469e-8       3.86644e-8         -1.0e6           -5.38462e5
     -1.53846e5   -38461.5         …       3.38462e6        1.50658e-7
  38461.5             -1.40385e6           1.50658e-7       3.88462e6

In [15]:
 # Imponemos condiciones de contorno de Dirichlet
 # U1x=0 => U1=0
    K[1,:] .= 0
    K[:,1] .= 0
    K[1,1] =1
 # U2x=0 => U3=0
    K[3,:] .= 0
    K[:,3] .= 0
    K[3,3] =1
 # U2y=0 => U4=0
    K[4,:] .= 0
    K[:,4] .= 0
    K[4,4] =1

display(K)

12×12 Matrix{Num}:
 1       0           0  0  …       0                0
 0       1.25481e6   0  0     -38461.5             -1.40385e6
 0       0           1  0          0                0
 0       0           0  1          0                0
 0    9615.38        0  0         -1.53846e5   -38461.5
 0       3.50962e5   0  0  …   38461.5             -1.40385e6
 0      -4.61538e5   0  0         -1.53846e6        1.0e6
 0      -2.69231e5   0  0          1.0e6           -5.38462e5
 0       1.15469e-8  0  0         -1.53846e6       -1.0e6
 0       3.86644e-8  0  0         -1.0e6           -5.38462e5
 0  -38461.5         0  0  …       3.38462e6        1.50658e-7
 0      -1.40385e6   0  0          1.50658e-7       3.88462e6

In [16]:
#= \(A,B)
Matrix division using a polyalgorithm. For input matrices A and B, the result X is such that A*X == B when A is square. 
The solver that is used depends upon the structure of A. If A is upper or lower triangular (or diagonal), no factorization 
of A is required and the system is solved with either forward or backward substitution. For non-triangular square matrices, 
an LU factorization is used.
=#

In [17]:
# CASO 1. F1 = 500kgf ; F2 = 0kgf

R1 =[0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0] # Fuerzas nodales en kgf.
U1 = K \ R1 # U1 --> Desplazamientos nodales.
display(U1)
E1 = B * U1
σ1 = C * E1

# Matriz de deformaciones nodales
Enodal1 = zeros(Num,3,6)
CoordEl = [0 0; 1 0; 0 1; 1/2 0; 1/2 1/2; 0 1/2]

for i in 1:6, j in 1:3
    Enodal1[j,i] = substitute(E1[j], Dict(r => CoordEl[i,1], s => CoordEl[i,2]),)
end
σNodal1 = C * Enodal1
display(Enodal1)
display(σNodal1)

12-element Vector{Num}:
 0.0
 0.004464285714286873
 0.0
 0.0
 0.00482142857142951
 0.005178571428572778
 4.464285714207837e-6
 0.002392857142857827
 0.002415178571429064
 0.002428571428572098
 0.001321428571428712
 0.004821428571429841

3×6 Matrix{Num}:
 -7.14286e-5   -0.000809524  -7.14286e-5   …  -0.000440476  -7.14286e-5
  0.000238095  -0.000190476   0.000238095      2.38095e-5    0.000238095
 -0.0012381    -3.67545e-17   0.0012381        0.000619048  -4.91144e-17

3×6 Matrix{Num}:
    -3.6863e-11  -2000.0          …  -1000.0  -1000.0    1.04762e-10
   500.0         -1000.0              -250.0   -250.0  500.0
 -1000.0            -2.96863e-11      -500.0    500.0   -3.96693e-11

In [18]:
# CASO 2. F1 = 0kgf ; F2 = 500kgf

R2 =[0, 0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0] # Fuerzas nodales en kgf.
U2 = K \ R2 # U3 --> Desplazamientos nodales.
E2 = B * U2
σ2 = C * E2

# Matriz de deformaciones nodales
Enodal2 = zeros(Num,3,6)
CoordEl = [0 0; 1 0; 0 1; 1/2 0; 1/2 1/2; 0 1/2]

for i in 1:6, j in 1:3
    Enodal2[j,i] = substitute(E2[j], Dict(r => CoordEl[i,1], s => CoordEl[i,2]),)
end
σNodal2 = C * Enodal2
display(Enodal2)
display(σNodal2)

3×6 Matrix{Num}:
 -0.00102381  -0.000738095  0.000738095  …   8.84709e-17  -0.000142857
  0.00052381  -0.000428571  0.000428571     -4.64039e-17   0.00047619
 -0.0012381   -0.0012381    0.0012381        1.17528e-16   1.17094e-17

3×6 Matrix{Num}:
 -2000.0  -2000.0  2000.0  -2000.0   1.72038e-10     2.85922e-11
   500.0  -1500.0  1500.0   -500.0  -4.58367e-11  1000.0
 -1000.0  -1000.0  1000.0  -1000.0   9.49261e-11     9.45758e-12

In [19]:
# CASO 3. F1 = 500kgf ; F2 = 500kgf

R3 =[0, 0, 0, 0, 500, 500, 0, 0, 0, 0, 0, 0] # Fuerzas nodales en kgf.
U3 = K \ R3 # U3 --> Desplazamientos nodales.
E3 = B * U3
σ3 = C * E3

# Matriz de deformaciones nodales
Enodal3 = zeros(Num,3,6)
CoordEl = [0 0; 1 0; 0 1; 1/2 0; 1/2 1/2; 0 1/2]

for i in 1:6, j in 1:3
    Enodal3[j,i] = substitute(E3[j], Dict(r => CoordEl[i,1], s => CoordEl[i,2]),)
end
σNodal3 = C * Enodal3
display(Enodal3)
display(σNodal3)

3×6 Matrix{Num}:
 -0.00109524   -0.00154762   0.000666667  …  -0.000440476  -0.000214286
  0.000761905  -0.000619048  0.000666667      2.38095e-5    0.000714286
 -0.00247619   -0.0012381    0.00247619       0.000619048  -3.55618e-17

3×6 Matrix{Num}:
 -2000.0  -4000.0  2000.0  -3000.0  -1000.0     1.33753e-10
  1000.0  -2500.0  2000.0   -750.0   -250.0  1500.0
 -2000.0  -1000.0  2000.0  -1500.0    500.0    -2.8723e-11

In [20]:
# Deformaciones en los puntos de Gauss.
#Caso 1.
EGauss1 = zeros(Num,3,7)

for i in 1:7, j in 1:3
    EGauss1[j,i] = substitute(E1[j], Dict(r => GL7T[i,1], s => GL7T[i,2]),)
end
EGauss1[3,:] = EGauss1[3,:]
display(EGauss1)

3×7 Matrix{Num}:
 -0.000146188  -0.000660006  -0.000146188  …  -0.000115505  -0.00031746
  0.000194687  -0.000103659   0.000194687      0.000212503   9.52381e-5
 -0.000861888   8.47304e-17   0.000861888     -4.79217e-17  -4.52112e-17

In [21]:
SigmaGauss1 = C * EGauss1

3×7 Matrix{Num}:
 -202.573  -1594.85         -202.573  …  -119.432       -666.667
  348.07    -696.14          348.07       410.426          1.81615e-11
 -696.14       6.84361e-11   696.14        -3.8706e-11    -3.65168e-11

In [22]:
# Deformaciones en los puntos de Gauss.
#Caso 2.
EGauss2 = zeros(Num,3,7)

for i in 1:7, j in 1:3
    EGauss2[j,i] = substitute(E2[j], Dict(r => GL7T[i,1], s => GL7T[i,2]),)
end
EGauss2[3,:] = EGauss2[3,:]
display(EGauss2)

3×7 Matrix{Num}:
 -0.000816413  -0.000617516  0.00041012   …  -0.000178402  -0.00034127
  0.0004177    -0.000245291  0.000351401      0.000422162   0.000174603
 -0.000987291  -0.000987291  0.000736486     -7.39339e-5   -0.000412698

In [23]:
SigmaGauss2 = C * EGauss2

3×7 Matrix{Num}:
 -1594.85   -1594.85   1189.71   -1761.14   -119.432   -119.432   -666.667
   398.713   -993.567  1094.85    -380.568    29.8579   850.71     166.667
  -797.427   -797.427   594.854   -880.568   -59.7159   -59.7159  -333.333

In [24]:
# Deformaciones en los puntos de Gauss.
#Caso 3.
EGauss3 = zeros(Num,3,7)

for i in 1:7, j in 1:3
    EGauss3[j,i] = substitute(E3[j], Dict(r => GL7T[i,1], s => GL7T[i,2]),)
end
EGauss3[3,:] = EGauss3[3,:]
display(EGauss3)

3×7 Matrix{Num}:
 -0.000962601  -0.00127752   0.000263932  …  -0.000293907  -0.00065873
  0.000612387  -0.00034895   0.000546087      0.000634665   0.000269841
 -0.00184918   -0.000987291  0.00159837      -7.39339e-5   -0.000412698

In [25]:
SigmaGauss3 = C * EGauss3

3×7 Matrix{Num}:
 -1797.43   -3189.71    987.135  -2701.42   -1059.72   -238.863   -1333.33
   746.784  -1689.71   1442.92    -585.781   -175.355  1261.14      166.667
 -1493.57    -797.427  1290.99   -1290.99     350.71    -59.7159   -333.333